In [7]:
import pandas as pd


In [8]:
data = pd.read_excel("August-2024-Primary-Rota1.xlsx")#Transform the data
data.head()

,Unnamed: 0,,Unnamed: 2,Unnamed: 3,.1,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,.2,...,Unnamed: 127,Unnamed: 128,Unnamed: 129,Unnamed: 130,Unnamed: 131,Unnamed: 132,Unnamed: 133,Unnamed: 134,Unnamed: 135,Unnamed: 136
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Start date of the Month,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,0,0,0,Day,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,No,Name of Patient,Patient Number,Type of Shift,Total Shifts,NaN,NaN,1-August,0.0,...,0.0,31-August,0.0,31-August,0.0,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,1-10,11-20,21-31,Day,NaN,...,NaN,Day,NaN,Night,NaN,NaN,NaN,NaN,Client ID,sum
4,NaN,1,Sushila Gailaya,32,24 Hours,10,11,13,NaN,NaN,...,1000.0,NaN,NaN,Crg Gladys chepkirui - 1000,1000.0,NaN,32,33000.0,1,194000


In [9]:
import pandas as pd

def data_transform(data):
    """
    Transform the data by merging the second and third rows to create column headers.
    """
    # Merge the second and third rows for column headers
    first_row = data.iloc[2].astype(str).fillna('')  # Handle NaNs as empty strings
    second_row = data.iloc[3].astype(str).fillna('')
    column_row = first_row + '-' + second_row
    column_row = column_row.str.strip('-')  # Remove trailing/leading dashes
    column_row = column_row.str.replace('-nan', '', regex=False).str.replace('nan-', '', regex=False)  # Remove '-nan' or 'nan-'
    column_row = column_row.str.strip()  # Strip extra spaces
    data.columns = column_row  # Assign new headers
    data = data.iloc[4:].reset_index(drop=True)  # Skip header rows
    return data

# Load the Excel file
file_path = 'August-2024-Primary-Rota1.xlsx'
data = pd.read_excel(file_path)

# Transform the data
data = data_transform(data)

# Debug: Check columns after transformation
print("Transformed columns:", data.columns)

# Identify the actual column name for "Name of Patient"
patient_column = [col for col in data.columns if "Name of Patient" in col]
if not patient_column:
    raise KeyError("Column 'Name of Patient' not found in the transformed DataFrame.")
patient_column = patient_column[0]  # Use the identified column

# Drop unwanted columns if present
columns_to_drop = ['Total Shifts', 'Type of Shift', 'Patient Number', 'Client ID', 'sum']
data.drop(columns=[col for col in columns_to_drop if col in data.columns], inplace=True, errors='ignore')

# Fill missing 'Name of Patient' values
data[patient_column] = data[patient_column].ffill()

# Identify columns with dates for further processing
date_columns = [col for col in data.columns if '-' in col and any(char.isdigit() for char in col)]
if not date_columns:
    raise ValueError("No date-related columns found in the dataset.")

# Create a mapping for new column names (e.g., 1-September-Day -> 2024_09)
month_mapping = {
    'January': '01', 'February': '02', 'March': '03', 'April': '04',
    'May': '05', 'June': '06', 'July': '07', 'August': '08',
    'September': '09', 'October': '10', 'November': '11', 'December': '12'
}

def rename_date_column(col):
    try:
        day, month, _ = col.split('-')
        month_num = month_mapping.get(month, month)
        return f"2024_{month_num}"
    except ValueError:
        return col  # Return the column unchanged if it doesn't match the expected pattern

# Rename date-related columns
new_date_column_names = {col: rename_date_column(col) for col in date_columns}
data.rename(columns=new_date_column_names, inplace=True)

# Convert to long format for caregiver data
crg_columns = [col for col in data.columns if col.startswith('2024_')]
long_df = data.melt(
    id_vars=[patient_column],
    value_vars=crg_columns,
    var_name='Month',
    value_name='Caregiver Name'
)

# Clean the long-form DataFrame
long_df.dropna(subset=['Caregiver Name'], inplace=True)
long_df['Month'] = long_df['Month'].str.split('_').str[1]  # Extract the month part

# Calculate attendance counts
attendance_counts = long_df.groupby(
    [patient_column, 'Month', 'Caregiver Name']
).size().reset_index(name='Attendance Count')

# Aggregate for the final output
final_output_df = attendance_counts.groupby(
    [patient_column, 'Month'], as_index=False
).agg({
    'Caregiver Name': lambda x: ', '.join(map(str, set(x))),  # Ensure all elements are strings
    'Attendance Count': list  # Convert counts to a list
})

# Save to a CSV file and print
output_path = 'August-2024-Primary-Rota123.csv'
final_output_df.to_csv(output_path, index=False)
print(final_output_df)


Transformed columns: Index(['nan', 'No', 'Name of Patient', 'Patient Number', 'Type of Shift',
       'Total  Shifts-1-10', '11-20', '21-31', '1-August-Day', '0.0',
       ...
       '0.0', '31-August-Day', '0.0', '31-August-Night', '0.0', 'nan', 'nan',
       'nan', 'Client ID', 'sum'],
      dtype='object', length=137)
           Name of Patient Month  \
0      Aarti/Hasmati devan    08   
1      Aarti/Hasmati devan     1   
2     Abuod Abdalla/Dalila    08   
3     Abuod Abdalla/Dalila     1   
4    Amrat Shah/Niraj Shah    08   
..                     ...   ...   
215         Vishan/vasanti     1   
216           Zarina Nanji    08   
217           Zarina Nanji     1   
218          Zulekha Samji    08   
219          Zulekha Samji     1   

                                        Caregiver Name Attendance Count  
0    Crg Felistas Wambui - 1000, Crg Naomi Paul - 8...       [1, 3, 27]  
1                                                   10              [1]  
2    Crg Nora Mukuhi K